# Rabi Lenght

In [1]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

2023.01.10 11:54:28.330 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is /home/admin/Juan/qibolab/examples/laboneq_output/log
2023.01.10 11:54:28.331 laboneq.controller.controller  INFO   VERSION: laboneq 1.4.0
2023.01.10 11:54:28.332 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2023.01.10 11:54:28.425 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2023.01.10 11:54:29.039 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2023.01.10 11:54:29.042 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [22]:
from qibolab.pulses import PulseSequence

sequence = PulseSequence()
ro_pulse = Device.create_qubit_readout_pulse(0, start=0)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

2023.01.10 16:17:46.803 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 16:17:46.825 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 13
2023.01.10 16:17:46.825 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 2000
2023.01.10 16:17:46.826 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 16:17:47.573 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.10 16:17:47.753 laboneq.controller.util        CRITICAL The number of measurements (0) executed for device dev12146 on channel 0 does not match the number of measurements defined (1000). Probably the time between measurements or within a loop is too short. Please contact Zurich Instruments.


2023.01.10 16:17:47.753 laboneq.controller.util        CRITICAL The number of measurements (0) executed for device dev12146 on channel 0 does not match the number of measurements defined (1000). Probably the time between measurements or within a loop is too short. Please contact Zurich Instruments.


2023.01.10 16:17:47.758 laboneq.controller.controller  INFO   Step retry 2 of 3...
2023.01.10 16:17:49.223 laboneq.controller.controller  WARNING Stop conditions still not fulfilled after 1.000000 s, estimated execution time was 0.05 s. Continuing to the next step.
2023.01.10 16:17:49.285 laboneq.controller.controller  INFO   Finished near-time execution.
2.7470000935276317e-05 -1.630477875879376 -1.6384791251522303e-06 -2.7421092931182048e-05


In [3]:
sequence_exc = PulseSequence()
qd_pulse = Device.create_qubit_drive_pulse(0, start=0, duration=1000) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=1000)
sequence_exc.add(qd_pulse)
sequence_exc.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence_exc)
print(msr, phase, i , q)

2023.01.10 11:54:31.454 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 11:54:31.480 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 28
2023.01.10 11:54:31.480 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 6000
2023.01.10 11:54:31.480 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 11:54:32.332 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.10 11:54:32.936 laboneq.controller.controller  INFO   Finished near-time execution.
2.9862574382601845e-06 0.17001394553427934 2.9432027953420176e-06 5.052630929082512e-07


In [4]:
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np

def rabi_pulse_length(
    platform,
    qubit: int,
    pulse_duration_start,
    pulse_duration_end,
    pulse_duration_step,
    software_averages,
):
    platform.reload_settings()
    platform.apply_settings()

    data = DataUnits(name=f"data_q{qubit}", quantities={"Time": "ns"})

    sequence = PulseSequence()
    qd_pulse = platform.create_qubit_drive_pulse(qubit, start=0, duration=4)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=4)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_pulse_duration_range = np.arange(
        pulse_duration_start, pulse_duration_end, pulse_duration_step
    )

    # platform.instruments["shfqc_qa"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )
    # platform.instruments["shfqc_qc"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["qubit_freq"]
    #     - qd_pulse.frequency
    # )

    for _ in range(software_averages):
        for duration in qd_pulse_duration_range:
            qd_pulse.duration = duration
            ro_pulse.start = duration
    
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)
            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)

            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "Time[ns]": duration,
            }
            
            
            data.add(results)
            
        # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data
            

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For Rabi oscillations
def time_msr_phase(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "popt4",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("Time", "ns"),
            y=data.get_values("MSR", "uV"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.get_values("Time", "ns"),
            y=data.get_values("phase", "rad"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=2,
    )

    # # add fitting trace
    # if len(data) > 0 and len(data_fit) > 0:
    #     timerange = np.linspace(
    #         min(data.get_values("Time", "ns")),
    #         max(data.get_values("Time", "ns")),
    #         2 * len(data),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=timerange,
    #             y=rabi(
    #                 timerange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #                 data_fit.get_values("popt4"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.20,
    #             showarrow=False,
    #             text=f"Estimated {params[1]} is {data_fit.df[params[1]][0]:.3f} ns.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"Estimated {params[0]} is {data_fit.df[params[0]][0]:.1f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    # last part
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Time (ns)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Time (ns)",
        yaxis2_title="Phase (rad)",
    )
    return fig

In [6]:
def rabi_pulse_length_sequences(
    platform,
    qubit: int,
    pulse_duration_start,
    pulse_duration_end,
    pulse_duration_step,
):

    sequences = []

    # sequence = PulseSequence()
    # qd_pulse = platform.create_qubit_drive_pulse(qubit, start=0, duration=4)
    # ro_pulse = platform.create_qubit_readout_pulse(qubit, start=4)
    # sequence.add(qd_pulse)
    # sequence.add(ro_pulse)

    qd_pulse_duration_range = np.arange(
        pulse_duration_start, pulse_duration_end, pulse_duration_step
    )

    durations = []
    for duration in qd_pulse_duration_range:
        
        sequence = PulseSequence()
        qd_pulse = platform.create_qubit_drive_pulse(qubit, start=0, duration=duration)
        ro_pulse = platform.create_qubit_readout_pulse(qubit, start=duration)
        sequence.add(qd_pulse)
        sequence.add(ro_pulse)
        
        # qd_pulse.duration = duration
        # ro_pulse.start = duration

        sequences.append(sequence)
        durations.append(duration)
        
    return sequences, durations
            

In [7]:
Device.reload_settings()
Device.apply_settings()

In [12]:
sequences, durations = rabi_pulse_length_sequences(Device,
    qubit = 0,
    pulse_duration_start = 20,
    pulse_duration_end = 320,
    pulse_duration_step = 10,
)

msr, phase, i , q = Device.execute_pulse_sequences(sequences)
qubit = 0

data = DataUnits(name=f"data_q{qubit}", quantities={"Time": "ns"})

results = {
    "MSR[V]": msr,
    "i[V]": i,
    "q[V]": q,
    "phase[rad]": phase,
    "Time[ns]": durations,
}
                   
data.set(results)

time_msr_phase(data)

2023.01.10 11:56:17.040 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 11:56:17.775 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 559
2023.01.10 11:56:17.776 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 482880
2023.01.10 11:56:17.776 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 11:56:18.587 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.10 11:56:22.238 laboneq.controller.controller  INFO   Finished near-time execution.


/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning:

pint-pandas does not support magnitudes of int64. Converting magnitudes to float.



In [13]:
sequences, durations = rabi_pulse_length_sequences(Device,
    qubit = 0,
    pulse_duration_start = 20,
    pulse_duration_end = 320,
    pulse_duration_step = 10,
)

msr, phase, i , q = Device.execute_pulse_sequences_np(sequences)
qubit = 0

data = DataUnits(name=f"data_q{qubit}", quantities={"Time": "ns"})

results = {
    "MSR[V]": msr,
    "i[V]": i,
    "q[V]": q,
    "phase[rad]": phase,
    "Time[ns]": durations,
}
                   
data.set(results)

time_msr_phase(data)

2023.01.10 11:56:39.805 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 11:56:40.484 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 559
2023.01.10 11:56:40.485 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 482880
2023.01.10 11:56:40.485 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 11:56:41.280 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.10 11:56:44.969 laboneq.controller.controller  INFO   Finished near-time execution.


/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning:

pint-pandas does not support magnitudes of int64. Converting magnitudes to float.



Con sequences entre 100 y 75 rabis

In [ ]:
class Sweep:
    
    def __init__(self, qubit, pulse, param, start, stop, count):
        

In [19]:
def rabi_pulse_length_sequenceSweep(
    platform,
    qubit: int,
    pulse_duration_start,
    pulse_duration_end,
    count,
    parameter,
):


    sequence = PulseSequence()
    qd_pulse = platform.create_qubit_drive_pulse(qubit, start=0, duration=pulse_duration_start)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, qd_pulse.finish)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_pulse_duration_range = np.linspace(
            pulse_duration_start, pulse_duration_end, count
        )

    durations = []
    for duration in qd_pulse_duration_range:

        durations.append(duration)
        
    return sequence, durations, pulse_duration_start, pulse_duration_end, count, parameter

In [20]:
Device.reload_settings()
Device.apply_settings()

In [21]:
sequence, durations, start, stop, count, parameter = rabi_pulse_length_sequenceSweep(Device,
    qubit = 0,
    pulse_duration_start = 20,
    pulse_duration_end = 320,
    count = 30,
    parameter= "Lenght",
)

msr, phase, i , q = Device.execute_pulse_sequence_Sweep(sequence, start, stop, count, parameter)
qubit = 0

data = DataUnits(name=f"data_q{qubit}", quantities={"Time": "ns"})

results = {
    "MSR[V]": msr,
    "i[V]": i,
    "q[V]": q,
    "phase[rad]": phase,
    "Time[ns]": durations,
}
                   
data.set(results)

time_msr_phase(data)

2023.01.10 12:06:05.946 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 12:06:06.070 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 608
2023.01.10 12:06:06.070 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 63440
2023.01.10 12:06:06.070 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 12:06:06.955 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.10 12:06:08.909 laboneq.controller.controller  INFO   Finished near-time execution.


/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



Con sweeper tambien entre 100 y 75 Rabis

In [21]:
lo.show_pulse_sheet("Pulse_sheet",Device.exp)

2022.12.15 11:32:25.402 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to /home/admin/Juan/qibolab/examples/Pulse_sheet_2022-12-15-11-32-25.html


/home/admin/Juan/qibolab/examples/Pulse_sheet_2022-12-15-11-32-25.html

In [16]:
Device.reload_settings()
Device.apply_settings()

In [17]:
data = rabi_pulse_length(Device,
    qubit = 0,
    pulse_duration_start = 20,
    pulse_duration_end = 320,
    pulse_duration_step = 10,
    software_averages = 1,
)

time_msr_phase(data)

2023.01.10 12:03:23.900 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 12:03:23.925 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 28
2023.01.10 12:03:23.926 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 4048
2023.01.10 12:03:23.926 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 12:03:24.809 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.10 12:03:25.392 laboneq.controller.controller  INFO   Finished near-time execution.
2023.01.10 12:03:25.859 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.10 12:03:25.884 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 28
2023.01.10 12:03:25.884 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 4048
2023.01.10 12:03:25.884 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.10 12:03:26.

In [17]:
lo.show_pulse_sheet("Pulse_sheet",Device.exp)

2022.12.08 15:56:08.457 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to /home/admin/Juan/qibolab/examples/Pulse_sheet_2022-12-08-15-56-08.html


/home/admin/Juan/qibolab/examples/Pulse_sheet_2022-12-08-15-56-08.html

Check Pi-pulse

In [1]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA
from qibolab.pulses import PulseSequence

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

2022.12.09 11:35:39.776 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is /home/admin/Juan/qibolab/examples/laboneq_output/log
2022.12.09 11:35:39.777 laboneq.controller.controller  INFO   VERSION: laboneq 1.4.0
2022.12.09 11:35:39.777 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2022.12.09 11:35:39.884 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2022.12.09 11:35:40.403 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2022.12.09 11:35:40.549 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [2]:
Device.reload_settings()
Device.native_gates["single_qubit"][0]["RX"]["duration"] = 80
Device.apply_settings()

In [3]:
sequence = PulseSequence()
qd_pulse = Device.create_RX_pulse(0, start=0) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse.finish)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

2022.12.09 10:18:50.881 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 10:18:50.907 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 40
2022.12.09 10:18:50.908 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 4048
2022.12.09 10:18:50.908 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2022.12.09 10:18:51.840 laboneq.controller.controller  INFO   Starting near-time execution...
2022.12.09 10:18:52.876 laboneq.controller.controller  INFO   Finished near-time execution.
4.870261681736926e-05 -2.417817375168851 -3.649340732320096e-05 -3.225176129767863e-05


In [2]:
sequence = PulseSequence()
qd_pulse = Device.create_RX_pulse(0, start=0) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse.finish)
sequence.add(qd_pulse)
sequence.add(ro_pulse)

msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

sequence2 = PulseSequence()
qd2_pulse = Device.create_RX90_pulse(0, start=0) 
ro2_pulse = Device.create_qubit_readout_pulse(0, start=qd2_pulse.finish)
sequence2.add(qd2_pulse)
sequence2.add(ro2_pulse)

msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence2)
print(msr, phase, i , q)

sequences = [ sequence, sequence2]

msr, phase, i, q = Device.execute_pulse_sequences(sequences)
print(msr, phase, i , q)

2022.12.09 11:35:41.957 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 11:35:41.985 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 40
2022.12.09 11:35:41.985 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 4048
2022.12.09 11:35:41.985 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2022.12.09 11:35:42.935 laboneq.controller.controller  INFO   Starting near-time execution...
2022.12.09 11:35:43.963 laboneq.controller.controller  INFO   Finished near-time execution.
4.448650383460387e-05 -2.4250207764165754 -3.354557132185204e-05 -2.921889231502047e-05
2022.12.09 11:35:44.329 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 11:35:44.354 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 40
2022.12.09 11:35:44.354 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 4048
2022.12.09 11:35:44.355 labone

In [3]:
import laboneq.simple as lo
lo.show_pulse_sheet("Resonator Spectroscopy", Device.exp)

2022.12.09 11:36:00.913 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to /home/admin/Juan/qibolab/examples/Resonator Spectroscopy_2022-12-09-11-36-00.html


/home/admin/Juan/qibolab/examples/Resonator Spectroscopy_2022-12-09-11-36-00.html

In [20]:
Device.sequence_drive

[[PulseFunctional(function='gaussian', uid='drive0', amplitude=0.8, length=8e-08, pulse_parameters={'sigma': 0.4})],
 [PulseFunctional(function='gaussian', uid='drive0', amplitude=0.8, length=4e-08, pulse_parameters={'sigma': 0.4})]]

In [21]:
Device.sequence_readout

[[PulseFunctional(function='const', uid='readout_0_1', amplitude=0.1, length=1.0000000000000002e-06, pulse_parameters=None)],
 [PulseFunctional(function='const', uid='readout_1_1', amplitude=0.1, length=1.0000000000000002e-06, pulse_parameters=None)]]

------------------------------------

# Rabi Amplitude

Something wrong

In [ ]:
def rabi_pulse_amp(
    platform,
    qubit: int,
    Amp_start,
    Amp_end,
    Amp_step,
    software_averages,
):
    platform.reload_settings()
    platform.apply_settings()

    data = DataUnits(name=f"data_q{qubit}", quantities={"Amp": "A"})

    sequence = PulseSequence()
    qd_pulse = platform.create_RX_pulse(qubit, start=0)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=qd_pulse.finish)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_amp_range = np.arange(
        Amp_start, Amp_end, Amp_step
    )

    for _ in range(software_averages):
        for Amp in qd_amp_range:

            qd_pulse.amplitude = Amp

            # platform.reload_settings()
            
            # platform.native_gates["single_qubit"][qubit]["RX"]["amplitude"] = Amp 

            # platform.apply_settings()

            # sequence = PulseSequence()
            # qd_pulse = platform.create_RX_pulse(qubit, start=0)
            # ro_pulse = platform.create_qubit_readout_pulse(qubit, start=qd_pulse.finish)
            # sequence.add(qd_pulse)
            # sequence.add(ro_pulse)
            
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)
            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)

            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "Amp[A]": Amp,
            }
            
            
            data.add(results)
            
        # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def amplitude_msr_phase(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = DataUnits()

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("amplitude", "dimensionless"),
            y=data.get_values("MSR", "uV"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.get_values("amplitude", "dimensionless"),
            y=data.get_values("phase", "rad"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=2,
    )

    # # add fitting trace
    # if len(data) > 0 and len(data_fit) > 0:
    #     amplituderange = np.linspace(
    #         min(data.get_values("amplitude", "dimensionless")),
    #         max(data.get_values("amplitude", "dimensionless")),
    #         2 * len(data),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=amplituderange,
    #             y=rabi(
    #                 amplituderange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #                 data_fit.get_values("popt4"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"Estimated {params[0]} is {data_fit.df[params[0]][0]:.3f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"Estimated {params[1]} is {data_fit.df[params[1]][0]:.4f}",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Amplitude (dimensionless)",
        yaxis_title="MSR (uV)",
    )
    return fig

In [ ]:
def rabi_pulse_Amp_sequenceSweep(
    platform,
    qubit: int,
    start,
    end,
    count,
    parameter,
):


    sequence = PulseSequence()
    qd_pulse = platform.create_qubit_drive_pulse(qubit, start=0, duration=pulse_duration_start)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, qd_pulse.finish)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_pulse_duration_range = np.linspace(
            pulse_duration_start, pulse_duration_end, count
        )

    durations = []
    for duration in qd_pulse_duration_range:

        durations.append(duration)
        
    return sequence, durations, pulse_duration_start, pulse_duration_end, count, parameter

In [ ]:
sequence, Amps, start, stop, count, parameter = rabi_pulse_length_sequenceSweep(Device,
    qubit = 0,
    pulse_duration_start = .5,
    pulse_duration_end = .9,
    count = 5,
    parameter= "Amp",
)

msr, phase, i , q = Device.execute_pulse_sequence_Sweep(sequence, start, stop, count, parameter)
qubit = 0

data = DataUnits(name=f"data_q{qubit}", quantities={"Time": "ns"})

results = {
    "MSR[V]": msr,
    "i[V]": i,
    "q[V]": q,
    "phase[rad]": phase,
    "Time[ns]": Amps,
}
                   
data.set(results)

time_msr_phase(data)

In [ ]:
data = rabi_pulse_amp(Device,
    qubit = 0,
    Amp_start = 0.8,
    Amp_end = 1,
    Amp_step = 0.3,
    software_averages = 1,
)

amplitude_msr_phase(data)

---------------------------------------------------

# Rabi Power

Already at max drive_range

In [ ]:
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np

def rabi_pulse_drive_range(
    platform,
    qubit: int,
    drive_range_start,
    drive_range_end,
    drive_range_step,
    software_averages,
):
    platform.reload_settings()
    platform.apply_settings()

    data = DataUnits(name=f"data_q{qubit}", quantities={"Drive_range": "dimensionless"})

    sequence = PulseSequence()
    qd_pulse = platform.create_qubit_RX_pulse(qubit, start=0)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=qd_pulse.finish)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_pulse_drive_range_range = np.arange(
        drive_range_start, drive_range_end,drive_range_step
    )

    # platform.instruments["shfqc_qa"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )
    # platform.instruments["shfqc_qc"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["qubit_freq"]
    #     - qd_pulse.frequency
    # )

    for _ in range(software_averages):
        for drive_range in qd_pulse_drive_range_range:
            
            platform.reload_settings()
            
            platform.instruments["shfqc_qc"]["settings"]["drive_range"] = drive_range
            
            platform.apply_settings()
    
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)
            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)

            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "Drive_range[dimensionless]": drive_range,
            }
            
            
            data.add(results)
            
        # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data

In [ ]:
def gain_msr_phase(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "popt4",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("gain", "dimensionless"),
            y=data.get_values("MSR", "uV"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.get_values("gain", "dimensionless"),
            y=data.get_values("phase", "rad"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=2,
    )

    # # add fitting trace
    # if len(data) > 0 and len(data_fit) > 0:
    #     gainrange = np.linspace(
    #         min(data.get_values("gain", "dimensionless")),
    #         max(data.get_values("gain", "dimensionless")),
    #         2 * len(data),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=gainrange,
    #             y=rabi(
    #                 gainrange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #                 data_fit.get_values("popt4"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.20,
    #             showarrow=False,
    #             text=f"Estimated {params[1]} is {data_fit.df[params[1]][0]:.3f}",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"Estimated {params[0]} is {data_fit.df[params[0]][0]:.4f} uV",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Gain (dimensionless)",
        yaxis_title="MSR (uV)",
    )
    return fig
